In [1]:
from MySQL import MySQL_Run

MySQL_Run("show tables")

(('admin_roles',),
 ('admin_users',),
 ('bus_route_stations',),
 ('bus_routes',),
 ('bus_routes_total',),
 ('users',))

# Create Table

In [9]:
MySQL_Run("""
CREATE TABLE IF NOT EXISTS `bus_route_stations` (
  `route_id` INT UNSIGNED NOT NULL COMMENT '所屬路徑ID（對應 bus_routes_total.route_id）',
  `route_name` VARCHAR(255) NOT NULL COMMENT '路徑名稱',
  `direction` ENUM('去程','回程') NOT NULL COMMENT '路程方向',
  `stop_name` VARCHAR(255) NOT NULL COMMENT '站點名稱',
  `latitude` DOUBLE NOT NULL COMMENT '緯度',
  `longitude` DOUBLE NOT NULL COMMENT '經度',
  `eta_from_start` INT UNSIGNED DEFAULT NULL COMMENT '首站到此站的預估時間（分鐘）',
  `stop_order` INT UNSIGNED NOT NULL COMMENT '第幾站（排序用）',
  `address` VARCHAR(255) DEFAULT NULL COMMENT '站點地址',
  `status` TINYINT NOT NULL DEFAULT 0 COMMENT '狀態（0=正常/啟用，可自訂）',
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '建立時間',
  KEY `idx_route_id` (`route_id`),
  KEY `idx_route_name` (`route_name`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

""")

'Query executed.'

In [ ]:
MySQL_Run("""
CREATE TABLE IF NOT EXISTS `bus_routes_total` (
  `route_id` INT UNSIGNED NOT NULL AUTO_INCREMENT COMMENT '路徑ID（自動遞增、唯一）',
  `route_name` VARCHAR(255) NOT NULL COMMENT '路徑名稱',
  `stop_count` INT UNSIGNED NOT NULL COMMENT '站點數量',
  `direction` ENUM('單向', '雙向') NOT NULL COMMENT '單程或雙向',
  `start_stop` VARCHAR(255) DEFAULT NULL COMMENT '首站',
  `end_stop` VARCHAR(255) DEFAULT NULL COMMENT '末站',
  `status` TINYINT NOT NULL DEFAULT 0 COMMENT '狀態（0=正常/啟用，可自訂）',
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '建立時間',
  PRIMARY KEY (`route_id`),
  UNIQUE KEY `uq_route_name` (`route_name`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
""")
MySQL_Run("""
INSERT INTO `bus_routes_total` (`route_name`, `stop_count`, `direction`, `start_stop`, `end_stop`, `status`)
VALUES
  ('市民小巴5(洽公直達線)', 8, '雙向', '花蓮轉運站', '花蓮縣政府 正門', 0),
  ('市民小巴6(醫療照護線)', 8, '雙向', '花蓮轉運站', '門諾醫院', 0),
  ('市民小巴7(市區夜環線) 單向循環線', 11, '單向', '花蓮轉運站', '花蓮轉運站', 0),
  ('市民小巴-行動遊花蓮 單向循環線', 24, '單向', '花蓮轉運站', '七星柴魚博物館', 0);
""")

'Query executed.'

# Instert

In [ ]:
import pandas as pd

df = pd.read_csv(r"D:\2025Cody\Github\H_Bus\client\src\data\Station.csv")
values = []
for _, row in df.iterrows():
    sql = f"({int(row['路徑ID'])}, '{row['路徑名稱']}', '{row['路程']}', '{row['站點']}', {row['去程緯度']}, {row['去程經度']}, {int(row['首站到此站時間'])}, {int(row['站次'])}, '{row['位置']}', {int(row['狀態'])})"
    values.append(sql)

insert_sql = "INSERT INTO `bus_route_stations` (`route_id`, `route_name`, `direction`, `stop_name`, `latitude`, `longitude`, `eta_from_start`, `stop_order`, `address`, `status`) VALUES\n" + ",\n".join(values) + ";"

print(insert_sql)
MySQL_Run(insert_sql)

# Read


In [ ]:
Infor = {
    "host": "10.53.1.3",
    "user": "root",
    "port": 7000,
    "password": "dh9j2(HU#s9h",
    "database": "bus_system",
    "table": "IOT_DPM_C530E"
}

Result = MySQL_Run("select * from bus_routes_total", Parameter = Infor)

Result

In [8]:
import pandas as pd 
Result = MySQL_Run("select * from bus_routes_total", Parameter = Infor)
COLUMNS = MySQL_Run("SHOW COLUMNS FROM bus_routes_total", Parameter = Infor)
cols = [c[0] for c in COLUMNS]
df = pd.DataFrame(Result, columns=cols)
df

,route_id,route_name,stop_count,direction,start_stop,end_stop,status,created_at
0,1,市民小巴5(洽公直達線),8,雙向,花蓮轉運站,花蓮縣政府 正門,0,2025-09-02 14:27:03
1,2,市民小巴6(醫療照護線),8,雙向,花蓮轉運站,門諾醫院,0,2025-09-02 14:27:03
2,3,市民小巴7(市區夜環線) 單向循環線,11,單向,花蓮轉運站,花蓮轉運站,0,2025-09-02 14:27:03


In [10]:
import pandas as pd 

Result = MySQL_Run("select * from bus_route_stations where route_id = 3", Parameter = Infor)
COLUMNS = MySQL_Run("SHOW COLUMNS FROM bus_route_stations", Parameter = Infor)
cols = [c[0] for c in COLUMNS]
df = pd.DataFrame(Result, columns=cols)
df

,route_id,route_name,direction,stop_name,latitude,longitude,eta_from_start,stop_order,address,status,created_at
0,3,市民小巴7(市區夜環線)-單向循環線,去程,花蓮轉運站,23.993020,121.603219,0,1,NONE,0,2025-09-02 14:24:30
1,3,市民小巴7(市區夜環線)-單向循環線,去程,勤天商旅,23.993665,121.604872,0,1,NONE,0,2025-09-02 14:24:30
2,3,市民小巴7(市區夜環線)-單向循環線,去程,市立圖書館,23.989980,121.601315,5,2,NONE,0,2025-09-02 14:24:30
3,3,市民小巴7(市區夜環線)-單向循環線,去程,麗星飯店,23.988384,121.600419,10,3,NONE,0,2025-09-02 14:24:30
4,3,市民小巴7(市區夜環線)-單向循環線,去程,花蓮又一村文創園區,23.977802,121.602481,15,4,NONE,0,2025-09-02 14:24:30
5,3,市民小巴7(市區夜環線)-單向循環線,去程,花蓮文創園區,23.975652,121.604519,20,5,NONE,0,2025-09-02 14:24:30
6,3,市民小巴7(市區夜環線)-單向循環線,去程,日出香榭大道1,23.975530,121.606394,25,6,NONE,0,2025-09-02 14:24:30
7,3,市民小巴7(市區夜環線)-單向循環線,去程,日出香榭大道2,23.974270,121.607381,30,7,NONE,0,2025-09-02 14:24:30
8,3,市民小巴7(市區夜環線)-單向循環線,去程,東大門夜市,23.972523,121.609228,35,8,NONE,0,2025-09-02 14:24:30
9,3,市民小巴7(市區夜環線)-單向循環線,去程,中華路,23.976161,121.608885,40,9,NONE,0,2025-09-02 14:24:30
